# part1
Bitwise logic gates are new to me, perhaps I do want to try and emulate them to understand how to get from the sample input to the sample output, i.e., from this,

```
123 -> x
456 -> y
x AND y -> d
x OR y -> e
x LSHIFT 2 -> f
y RSHIFT 2 -> g
NOT x -> h
NOT y -> i
```

, to this,

```
d: 72
e: 507
f: 492
g: 114
h: 65412
i: 65079
x: 123
y: 456
```

1. I need a way to parse the input and convert into the BitewiseOperator functions which are part of Python

In [1]:
import operator

In [2]:
day7_example_circuit = [] 
day7_example_inputs = "inputs/day7_example.txt"

total_no_instructions = 0
with open(day7_example_inputs) as file:
    day7_example_circuit = file.readlines()
    total_no_instructions = len(day7_example_inputs)

In [3]:
bitwise_operators = {
    "AND": operator.and_,
    "OR": operator.or_,
    "LSHIFT": operator.lshift,
    "RSHIFT": operator.rshift,
    "NOT": operator.inv
}

In [4]:
wire_values = {}

for instruction in day7_example_circuit:
    instruct_split = instruction.split("->")
    wire = instruct_split[-1].strip()
    print(instruction)
    if any(op in instruct_split[0] for op in bitwise_operators.keys()):
        instruct = instruct_split[0].strip().split(" ")

        if "NOT" in instruct:
            operator_instruct = bitwise_operators[instruct[0]]
            wire1 = wire_values[instruct[1]]
            wire_values[wire] = operator_instruct(int(wire1)) + 65535 + 1 # Can only be a number between 0 and 65535
        else:
            wire1 = wire_values[instruct[0]]
            try:
                wire2 = wire_values[instruct[-1]]
            except:
                wire2 = instruct[-1]
            wire3 = wire
            operator_instruct = bitwise_operators[instruct[1]]            
            wire_values[wire] = operator_instruct(int(wire1), int(wire2))
            
    else:
        # If no operator, assume straight assignment
        wire_values[wire] = int(instruct_split[0])

print(dict(sorted(wire_values.items())))

123 -> x

456 -> y

x AND y -> d

x OR y -> e

x LSHIFT 2 -> f

y RSHIFT 2 -> g

NOT x -> h

NOT y -> i
{'d': 72, 'e': 507, 'f': 492, 'g': 114, 'h': 65412, 'i': 65079, 'x': 123, 'y': 456}


In [5]:
# Now testing the code on the full data sample
day7_circuit = [] 
day7_inputs = "inputs/day7.txt"

total_no_instructions = 0
with open(day7_inputs) as file:
    day7_circuit = file.readlines()
    total_no_instructions = len(day7_inputs)

In [6]:
# Sort list of instructions so basic assignments are first: 
split_instructions = [item.strip().split(" ") for item in day7_circuit]
# sorted_instructions = sorted(split_instructions, key=operator.itemgetter(-1))

In [7]:
# Initial sort by the length, initial value assigning will be easier
sorted_instructions = sorted(split_instructions, key=len)

wire_values = {}

# Sort through the other instructions to see if they can be applied
while sorted_instructions:
    for instruction in sorted_instructions:
        wire = instruction[-1].strip()
    
        if not "NOT" in instruction:
            # Perform the "easy" assign instructions first
            wire1 = instruction[0]
            wire2 = instruction[2]
            if len(instruction) == 3:
                if instruction[0].isnumeric():
                    wire_values[wire] = int(wire1)
                    sorted_instructions.remove(instruction)
                else:
                    if wire1 in wire_values.keys():
                        wire_values[wire2] = wire_values[wire1]
                        sorted_instructions.remove(instruction)
            elif any(shift in instruction for shift in ["LSHIFT", "RSHIFT"]):
                # Sort through xSHIFT instructions to see if we can assign more values
                shift_value = instruction[2]
                operator_instruct = bitwise_operators[instruction[1]]
                if wire1 in wire_values:
                    wire1 = wire_values[wire1]
                    wire_values[wire] = operator_instruct(int(wire1), int(shift_value))
                    sorted_instructions.remove(instruction)
            elif wire1 in wire_values.keys() and wire2 in wire_values.keys():
                wire1 = wire_values[wire1]
                wire2 = wire_values[wire2]
                operator_instruct = bitwise_operators[instruction[1]]
                wire_values[wire] = operator_instruct(int(wire1), int(wire2))
                sorted_instructions.remove(instruction)
            elif wire2 in wire_values.keys() and wire1.isnumeric():                
                value1 = wire1
                wire2 = wire_values[wire2]
                operator_instruct = bitwise_operators[instruction[1]]
                wire_values[wire] = operator_instruct(int(value1), int(wire2))
                sorted_instructions.remove(instruction)
        elif "NOT" in instruction:
            wire1 = instruction[1]
            if wire1 in wire_values.keys():
                operator_instruct = bitwise_operators[instruction[0]]
                wire1 = wire_values[wire1]
                wire_values[wire] = operator_instruct(int(wire1)) + 65535 + 1 
                sorted_instructions.remove(instruction)
        
    #print("remaining_instructions", len(sorted_instructions))
    #print(sorted_instructions)
    #print(dict(sorted(wire_values.items())))

In [8]:
# Returning the result for a:
wire_a = wire_values["a"]
print("a:", wire_a)

a: 956


# part2
Part 2's instructions are:

> Now, take the signal you got on wire a, override wire b to that signal, and reset the other wires (including wire a). What new signal is ultimately provided to wire a?

In [11]:
sorted_instructions = sorted(split_instructions, key=len)
wire_values = {}

wire_values["b"] = wire_a

while sorted_instructions:
    for instruction in sorted_instructions:
        wire = instruction[-1].strip()
    
        if not "NOT" in instruction:
            # Perform the "easy" assign instructions first
            wire1 = instruction[0]
            wire2 = instruction[2]
 
            if len(instruction) == 3:
                if instruction[0].isnumeric():
                    if not wire == "b":
                        wire_values[wire] = int(wire1)
                        sorted_instructions.remove(instruction)
                    else: 
                        # Remove the existing instruction for b
                        sorted_instructions.remove(instruction)
                else:
                    if wire1 in wire_values.keys():
                        wire_values[wire2] = wire_values[wire1]
                        sorted_instructions.remove(instruction)
            elif any(shift in instruction for shift in ["LSHIFT", "RSHIFT"]):
                # Sort through xSHIFT instructions to see if we can assign more values
                shift_value = instruction[2]
                operator_instruct = bitwise_operators[instruction[1]]
                if wire1 in wire_values:
                    wire1 = wire_values[wire1]
                    wire_values[wire] = operator_instruct(int(wire1), int(shift_value))
                    sorted_instructions.remove(instruction)
            elif wire1 in wire_values.keys() and wire2 in wire_values.keys():
                wire1 = wire_values[wire1]
                wire2 = wire_values[wire2]
                operator_instruct = bitwise_operators[instruction[1]]
                wire_values[wire] = operator_instruct(int(wire1), int(wire2))
                sorted_instructions.remove(instruction)
            elif wire2 in wire_values.keys() and wire1.isnumeric():                
                value1 = wire1
                wire2 = wire_values[wire2]
                operator_instruct = bitwise_operators[instruction[1]]
                wire_values[wire] = operator_instruct(int(value1), int(wire2))
                sorted_instructions.remove(instruction)
        elif "NOT" in instruction:
            wire1 = instruction[1]
            if wire1 in wire_values.keys():
                operator_instruct = bitwise_operators[instruction[0]]
                wire1 = wire_values[wire1]
                wire_values[wire] = operator_instruct(int(wire1)) + 65535 + 1 
                sorted_instructions.remove(instruction)

    #print("remaining_instructions", len(sorted_instructions))
    #print(sorted_instructions)
    #print(dict(sorted(wire_values.items())))

In [12]:
# Returning the result for a:
wire_a = wire_values["a"]
print("a:", wire_a)

a: 40149
